In [67]:
# Standard library imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import timedelta

In [68]:
# Read the data file

df = pd.read_json('data/yelp_academic_dataset_business.json', lines=True)
print("Shape of the data frame:", df.shape)

Shape of the data frame: (150346, 14)


In [69]:
# Drop all records with missing values and irrelevant columns
df = df.dropna()
df = df.drop(columns=['name', 'address', 'city'])

print("Shape of the modified data frame:", df.shape)

Shape of the modified data frame: (117618, 11)


In [70]:
# Keep only businesses that are restaurants
df = df[df['categories'].str.contains('Restaurants')]

# Keep only businesses that are still open (not permanently closed)
df = df[df['is_open']==1]

# Drop the is_open column (irrelevant)
df = df.drop(columns='is_open')

print("Shape of the modified data frame:", df.shape)

Shape of the modified data frame: (31357, 10)


In [71]:
# Parse JSON data in attributes and hours columns to individual feature columns

df = df.join(pd.json_normalize(df['attributes']))
df = df.join(pd.json_normalize(df['hours']))

# Drop the attributes and hours columns containing JSON data
df = df.drop(columns=['attributes', 'hours'])

In [72]:
# Function to parse an hours string and return number of hours open
# e.g. 10:00-21:00 -> 11 hours

def parse_hours(day_hours_str):
    if pd.isna(day_hours_str):
        return 0
    
    time_endpoints = str(day_hours_str).split('-')

    if time_endpoints[0] == time_endpoints[1]:
        # 0:0-0:0
        return 0
    
    start_time = time.strptime(time_endpoints[0], "%H:%M")
    end_time = time.strptime(time_endpoints[1], "%H:%M")

    # account for edge cases in data where we have 10-1, which is technically 10am-1am
    et_hour = (24 + end_time.tm_hour) if end_time.tm_hour < start_time.tm_hour else end_time.tm_hour
    
    start_time_td = timedelta(hours=start_time.tm_hour, minutes=start_time.tm_min)
    end_time_td = timedelta(hours=et_hour, minutes=end_time.tm_min)

    duration = end_time_td - start_time_td

    return duration.total_seconds() / 3600

In [73]:
# Create new feature (total_open_hours) by combining all individual day hours

total_hours_arr = []
count_neg = 0
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for ind in df.index:
    total_hours = 0

    for day in days:
        day_hours_str = df[day][ind]
        day_hours = parse_hours(day_hours_str)
        total_hours += day_hours
    
    total_hours_arr.append(total_hours)

df['total_open_hours'] = total_hours_arr

# Drop all the individual day hours columns
df = df.drop(columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

# TODO show example of total_open_hours before dropping days?

In [74]:
# Remove all features except a handful

df = df.filter(['total_open_hours', 'RestaurantsTakeOut', 'RestaurantsDelivery', 'Alcohol', 'latitude', 'longitude', 'stars'])

In [75]:
# Impute missing values with false

df['RestaurantsTakeOut'] = df['RestaurantsTakeOut'].fillna('False')
df['RestaurantsDelivery'] = df['RestaurantsDelivery'].fillna('False')
df['Alcohol'] = df['Alcohol'].fillna('False')

In [76]:
# Convert alcohol column to true/false values only

def alcohol_tf(val):
    if 'beer_and_wine' in val or 'full_bar' in val:
        return True
    else :
        return False

df['Alcohol_TF'] = df['Alcohol'].apply(alcohol_tf)

In [77]:
# Replace all 'None' values with False
df.replace('None', 'False', inplace=True)

# Convert all string representations of T/F to boolean values
df.replace({'True': True, 'False': False}, inplace=True)

# Drop alcohol feature (already feature engineered it)
df.drop(columns=['Alcohol'], inplace = True)

# Rename alcohol T/F feature column
df = df.rename(columns={'Alcohol_TF':'Alcohol'})

# Reset the index to 0, 1, ... - it changed after all the drops and modifications
df = df.reset_index()

In [78]:
# Apply KNN to find k nearest restaurants (by latitude/longitude)
# Create new feature (avg_star_rating) averaging the star rating of the k nearest restaurants

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

location_df = df[['latitude', 'longitude', 'stars']]

stars = location_df['stars']
location_df = location_df.drop(columns='stars')

scaler = StandardScaler()
location_df = pd.DataFrame(scaler.fit_transform(location_df), columns=location_df.columns)

neigh = NearestNeighbors(n_neighbors=51, n_jobs=-1)

neigh.fit(location_df[['latitude', 'longitude']])

distances, indices = neigh.kneighbors(location_df[['latitude', 'longitude']])

for i in range(len(location_df)):
    location_df.loc[i, 'avg_star_rating'] = stars.iloc[indices[i]].mean()

In [79]:
# Add stacking feature to main df
df['stack_1'] = location_df['avg_star_rating']

# Drop latitude/longitude feature columns (no need anymore)
df.drop(columns=['latitude', 'longitude'], inplace=True)

# Reorder df columns
df = df[['RestaurantsTakeOut', 'RestaurantsDelivery', 'Alcohol', 'total_open_hours', 'stack_1', 'stars']]

In [81]:
# Final df for model looks like this

df.head()

,RestaurantsTakeOut,RestaurantsDelivery,Alcohol,total_open_hours,stack_1,stars
0,False,False,False,23.0,3.960784,4.0
1,True,True,True,53.0,3.225490,2.0
2,False,False,False,100.0,3.235294,1.5
3,True,True,False,36.0,3.931373,4.0
4,True,False,True,34.0,3.284314,2.5
